## 1. Load the VGG Model

In [1]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions, VGG16
import numpy as np
import os

conv_base = VGG16(weights = 'imagenet', include_top = False, input_shape=(150,150,3))

ModuleNotFoundError: No module named 'keras'

In [2]:
from keras import models
from keras import layers
conv_base.trainable = False
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.summary()

ModuleNotFoundError: No module named 'keras'

In [3]:
from keras import optimizers
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics=['acc'])

ModuleNotFoundError: No module named 'keras'

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = '/images/demo3/ponies/train'
validation_dir = '/images/demo3/ponies/validate'

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode = 'nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=4,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=4,
        class_mode='categorical')

ModuleNotFoundError: No module named 'keras'

In [5]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=10) 

NameError: name 'model' is not defined